In [39]:
import pandas as pd

original=pd.read_csv("./dataSource/final_data.csv")
original.head()

,id,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,price,minimum_months,maximum_months,distance_to_mrt,...,accommodates,picture_url,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,listing_url
0,71609,Tampines,East Region,1.34537,103.95887,Private room,4170,3,37,0.496792,...,3,https://a0.muscache.com/pictures/24453191/3580...,4.44,4.37,4.00,4.63,4.78,4.26,4.32,https://www.airbnb.com/rooms/71609
1,71896,Tampines,East Region,1.34754,103.95958,Private room,2100,3,37,0.678619,...,1,https://a0.muscache.com/pictures/2440674/ac4f4...,4.16,4.22,4.09,4.43,4.43,4.17,4.04,https://www.airbnb.com/rooms/71896
2,71903,Tampines,East Region,1.34531,103.96100,Private room,2430,3,37,0.400694,...,2,https://a0.muscache.com/pictures/568743/7bc623...,4.41,4.39,4.52,4.63,4.64,4.50,4.36,https://www.airbnb.com/rooms/71903
3,275343,Bukit Merah,Central Region,1.29015,103.80814,Private room,1650,3,33,0.580759,...,1,https://a0.muscache.com/pictures/miso/Hosting-...,4.40,4.16,4.26,4.47,4.42,4.53,4.63,https://www.airbnb.com/rooms/275343
4,275344,Bukit Merah,Central Region,1.28836,103.81144,Private room,2070,2,33,0.612490,...,1,https://a0.muscache.com/pictures/miso/Hosting-...,4.54,4.64,4.21,4.64,4.57,4.64,4.43,https://www.airbnb.com/rooms/275344


In [40]:
import re
type_to_add=original['name']
type_to_add = type_to_add.str.split(' in ').str[0]

studio_to_add=original['details']
studio_to_add = studio_to_add.apply(lambda x: 1 if 'Studio' in x else 0)

shared_bath_to_add=original['details']
shared_bath_to_add=shared_bath_to_add.apply(lambda x: 0.5 if 'Shared half-bath' in x else 0)

private_bath_to_add=original['details']
private_bath_to_add=private_bath_to_add.apply(lambda x: 0.5 if 'Private half-bath' in x else 0)

def parse_room_info(info):
    counts = {}  


    matches = re.findall(r'(\d+(\.\d+)?)\s*([\w\s]+)\b', info)
    for count, _, room_type in matches:
        counts[room_type.strip()] = float(count)

    return counts

parsed_room_info = original['details'].apply(parse_room_info)

parsed_df = pd.DataFrame(parsed_room_info.tolist(), index=original.index)


df = pd.concat([original, parsed_df], axis=1)
df.insert(1, 'types', type_to_add)

df[['bedrooms', 'bedroom', 'beds', 'bed', 'shared baths','private bath', 'shared bath', 'baths', 'bath']] = df[['bedrooms', 'bedroom', 'beds', 'bed', 'shared baths','private bath', 'shared bath', 'baths', 'bath']].fillna(0)
df['total_bedrooms'] = df['bedrooms'] + df['bedroom']
df['total_beds'] = df['beds'] + df['bed']
df['total_shared_baths'] = df['shared baths'] + df['shared bath']+shared_bath_to_add

df['private bath']=df['private bath']+ private_bath_to_add
df['total_baths'] = df['baths'] + df['bath']

def get_bath_type(row):
    if row['private bath'] != 0:
        return 'private'
    elif row['total_shared_baths'] != 0:
        return 'shared'
    elif row['total_baths'] != 0:
        return 'unknown'
    else:
        return 'none'

df['bath_type'] = df.apply(get_bath_type, axis=1)

df['total_baths'] =df['total_baths'] +df['total_shared_baths']+ df['private bath']


df = df.drop(columns=['bedrooms', 'bedroom', 'beds', 'bed', 'shared baths', 'shared bath', 'baths', 'bath','total_shared_baths','private bath'])
df.insert(loc=len(df.columns), column='studio', value=studio_to_add)

df.to_csv('./csv_output/data_merged_5002_v3.csv', index=False)

df.head()

,id,types,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,price,minimum_months,maximum_months,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,listing_url,total_bedrooms,total_beds,total_baths,bath_type,studio
0,71609,Villa,Tampines,East Region,1.34537,103.95887,Private room,4170,3,37,...,4.63,4.78,4.26,4.32,https://www.airbnb.com/rooms/71609,2.0,3.0,1.0,private,0
1,71896,Home,Tampines,East Region,1.34754,103.95958,Private room,2100,3,37,...,4.43,4.43,4.17,4.04,https://www.airbnb.com/rooms/71896,1.0,1.0,0.5,shared,0
2,71903,Home,Tampines,East Region,1.34531,103.96100,Private room,2430,3,37,...,4.63,4.64,4.50,4.36,https://www.airbnb.com/rooms/71903,1.0,2.0,0.5,shared,0
3,275343,Rental unit,Bukit Merah,Central Region,1.29015,103.80814,Private room,1650,3,33,...,4.47,4.42,4.53,4.63,https://www.airbnb.com/rooms/275343,1.0,1.0,2.0,shared,0
4,275344,Rental unit,Bukit Merah,Central Region,1.28836,103.81144,Private room,2070,2,33,...,4.64,4.57,4.64,4.43,https://www.airbnb.com/rooms/275344,1.0,1.0,2.5,shared,0


In [41]:
import pandas as pd
df1 = pd.read_csv('./dataSource/final_data.csv')
df2 = pd.read_csv('./csv_output/data_merged_5002_v3.csv')
columns_df1 = set(df1.columns)
columns_df2 = set(df2.columns)
different_columns = columns_df1.symmetric_difference(columns_df2)
print("different columns:", different_columns)


different columns: {'total_bedrooms', 'bath_type', 'types', 'studio', 'total_beds', 'total_baths'}


In [42]:
df1 = pd.read_csv('./csv_output/data_merged_5002_v2.csv')
df2 = pd.read_csv('./csv_output/data_merged_5002_v3.csv')
columns_df1 = set(df1.columns)
columns_df2 = set(df2.columns)
different_columns = columns_df1.symmetric_difference(columns_df2)
print("different columns:", different_columns)

different columns: {'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_value', 'review_scores_accuracy', 'review_scores_location', 'review_scores_communication', 'listing_url', 'review_scores_rating'}


In [43]:

result = df2[(df2['total_bedrooms'] == 0) & (df2['studio'] == 0)]
result

,id,types,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,price,minimum_months,maximum_months,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,listing_url,total_bedrooms,total_beds,total_baths,bath_type,studio
13,481789,Rental unit,Tampines,East Region,1.348160,103.932380,Private room,1110,6,24,...,0.0,0.0,0.0,0.0,https://www.airbnb.com/rooms/481789,0.0,1.0,1.0,unknown,0
15,642660,Rental unit,Rochor,Central Region,1.300960,103.852060,Private room,4860,6,24,...,0.0,0.0,0.0,0.0,https://www.airbnb.com/rooms/642660,0.0,1.0,2.0,unknown,0
18,733863,Rental unit,Serangoon,North-East Region,1.369260,103.873120,Private room,780,6,12,...,0.0,0.0,0.0,0.0,https://www.airbnb.com/rooms/733863,0.0,1.0,1.0,unknown,0
22,765579,Rental unit,Outram,Central Region,1.284100,103.842530,Private room,2070,3,12,...,4.0,4.5,4.0,3.0,https://www.airbnb.com/rooms/765579,0.0,1.0,2.0,shared,0
26,823571,Rental unit,Bedok,East Region,1.313130,103.914790,Private room,8100,3,37,...,0.0,0.0,0.0,0.0,https://www.airbnb.com/rooms/823571,0.0,1.0,0.0,none,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3264,873151521937178007,Rental unit,River Valley,Central Region,1.296649,103.837935,Entire home/apt,7050,0,12,...,0.0,0.0,0.0,0.0,https://www.airbnb.com/rooms/873151521937178007,0.0,1.0,1.0,unknown,0
3265,873151554401594108,Rental unit,River Valley,Central Region,1.296064,103.840301,Entire home/apt,7050,0,12,...,0.0,0.0,0.0,0.0,https://www.airbnb.com/rooms/873151554401594108,0.0,1.0,1.0,unknown,0
3266,873151589130962362,Rental unit,River Valley,Central Region,1.296581,103.838082,Entire home/apt,7350,0,12,...,2.0,1.0,2.0,1.0,https://www.airbnb.com/rooms/873151589130962362,0.0,1.0,1.0,unknown,0
3267,873151615789259032,Rental unit,River Valley,Central Region,1.296362,103.839518,Entire home/apt,7350,0,12,...,0.0,0.0,0.0,0.0,https://www.airbnb.com/rooms/873151615789259032,0.0,1.0,1.0,unknown,0


In [44]:
import pandas as pd

data_df = pd.read_csv('./dataSource/final_data.csv')
listing_df = pd.read_csv('./dataSource/listings.csv')

# 使用 isin() 和 ~ (NOT)操作符找到所有在data.csv中但不在listing.csv中的行
not_in_listing = data_df[~data_df['id'].isin(listing_df['id'])]
not_in_listing

,id,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,price,minimum_months,maximum_months,distance_to_mrt,...,accommodates,picture_url,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,listing_url


In [45]:
in_listing = listing_df[listing_df['id'].isin(data_df['id'])]
in_listing

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,71609,https://www.airbnb.com/rooms/71609,20230628212928,2023-06-29,city scrape,Villa in Singapore · ★4.44 · 2 bedrooms · 3 be...,For 3 rooms.Book room 1&2 and room 4<br /><br ...,NaN,https://a0.muscache.com/pictures/24453191/3580...,367042,...,4.78,4.26,4.32,NaN,f,5,0,5,0,0.14
1,71896,https://www.airbnb.com/rooms/71896,20230628212928,2023-06-28,city scrape,Home in Singapore · ★4.16 · 1 bedroom · 1 bed ...,<b>The space</b><br />Vocational Stay Deluxe B...,NaN,https://a0.muscache.com/pictures/2440674/ac4f4...,367042,...,4.43,4.17,4.04,NaN,f,5,0,5,0,0.17
2,71903,https://www.airbnb.com/rooms/71903,20230628212928,2023-06-29,city scrape,Home in Singapore · ★4.41 · 1 bedroom · 2 beds...,"Like your own home, 24hrs access.<br /><br /><...",Quiet and view of the playground with exercise...,https://a0.muscache.com/pictures/568743/7bc623...,367042,...,4.64,4.50,4.36,NaN,f,5,0,5,0,0.32
3,275343,https://www.airbnb.com/rooms/275343,20230628212928,2023-06-28,city scrape,Rental unit in Singapore · ★4.40 · 1 bedroom ·...,**IMPORTANT NOTES: READ BEFORE YOU BOOK! <br ...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,1439258,...,4.42,4.53,4.63,NaN,t,51,2,49,0,0.18
4,275344,https://www.airbnb.com/rooms/275344,20230628212928,2023-06-28,city scrape,Rental unit in Singapore · ★4.54 · 1 bedroom ·...,Lovely home for the special guest !<br /><br /...,Bus stop <br />Food center <br />Supermarket,https://a0.muscache.com/pictures/miso/Hosting-...,1439258,...,4.57,4.64,4.43,NaN,t,51,2,49,0,0.11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3438,920638892602895042,https://www.airbnb.com/rooms/920638892602895042,20230628212928,2023-06-28,city scrape,Rental unit in Singapore · ★New · 1 bedroom · ...,"Peaceful city view, night scape and green outl...",NaN,https://a0.muscache.com/pictures/miso/Hosting-...,24060343,...,NaN,NaN,NaN,NaN,f,2,0,2,0,NaN
3439,921632579190690942,https://www.airbnb.com/rooms/921632579190690942,20230628212928,2023-06-29,city scrape,Rental unit in Singapore · ★New · 3 bedrooms ·...,Geylang still reigns as a popular buzzing food...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,323944827,...,NaN,NaN,NaN,Authorised Serviced Apartment,f,9,9,0,0,NaN
3440,921860100684819207,https://www.airbnb.com/rooms/921860100684819207,20230628212928,2023-06-28,city scrape,Rental unit in Singapore · ★New · 1 bedroom · ...,Experience luxury in this 3-bedroom condo at R...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,9234861,...,NaN,NaN,NaN,NaN,f,2,1,1,0,NaN
3441,922639973995552700,https://www.airbnb.com/rooms/922639973995552700,20230628212928,2023-06-29,city scrape,Rental unit in Singapore · ★New · 1 bedroom · ...,Keep it simple at this peaceful and centrally-...,NaN,https://a0.muscache.com/pictures/34975b5f-4d66...,35666883,...,NaN,NaN,NaN,NaN,t,2,0,2,0,NaN


In [46]:
info_raw=in_listing[['id','name','listing_url','bathrooms_text','room_type','bedrooms','beds']]
info_raw

,id,name,listing_url,bathrooms_text,room_type,bedrooms,beds
0,71609,Villa in Singapore · ★4.44 · 2 bedrooms · 3 be...,https://www.airbnb.com/rooms/71609,1 private bath,Private room,NaN,3.0
1,71896,Home in Singapore · ★4.16 · 1 bedroom · 1 bed ...,https://www.airbnb.com/rooms/71896,Shared half-bath,Private room,NaN,1.0
2,71903,Home in Singapore · ★4.41 · 1 bedroom · 2 beds...,https://www.airbnb.com/rooms/71903,Shared half-bath,Private room,NaN,2.0
3,275343,Rental unit in Singapore · ★4.40 · 1 bedroom ·...,https://www.airbnb.com/rooms/275343,2 shared baths,Private room,NaN,1.0
4,275344,Rental unit in Singapore · ★4.54 · 1 bedroom ·...,https://www.airbnb.com/rooms/275344,2.5 shared baths,Private room,NaN,1.0
...,...,...,...,...,...,...,...
3438,920638892602895042,Rental unit in Singapore · ★New · 1 bedroom · ...,https://www.airbnb.com/rooms/920638892602895042,1 bath,Private room,NaN,1.0
3439,921632579190690942,Rental unit in Singapore · ★New · 3 bedrooms ·...,https://www.airbnb.com/rooms/921632579190690942,2 baths,Entire home/apt,3.0,3.0
3440,921860100684819207,Rental unit in Singapore · ★New · 1 bedroom · ...,https://www.airbnb.com/rooms/921860100684819207,1 bath,Private room,NaN,1.0
3441,922639973995552700,Rental unit in Singapore · ★New · 1 bedroom · ...,https://www.airbnb.com/rooms/922639973995552700,1.5 baths,Private room,1.0,1.0


In [47]:
info_raw.to_csv('./csv_output/info_raw.csv')

In [48]:
data = pd.read_csv('./csv_output/info_raw.csv')

# Show the first few rows of the dataset
data.head()

,Unnamed: 0,id,name,listing_url,bathrooms_text,room_type,bedrooms,beds
0,0,71609,Villa in Singapore · ★4.44 · 2 bedrooms · 3 be...,https://www.airbnb.com/rooms/71609,1 private bath,Private room,NaN,3.0
1,1,71896,Home in Singapore · ★4.16 · 1 bedroom · 1 bed ...,https://www.airbnb.com/rooms/71896,Shared half-bath,Private room,NaN,1.0
2,2,71903,Home in Singapore · ★4.41 · 1 bedroom · 2 beds...,https://www.airbnb.com/rooms/71903,Shared half-bath,Private room,NaN,2.0
3,3,275343,Rental unit in Singapore · ★4.40 · 1 bedroom ·...,https://www.airbnb.com/rooms/275343,2 shared baths,Private room,NaN,1.0
4,4,275344,Rental unit in Singapore · ★4.54 · 1 bedroom ·...,https://www.airbnb.com/rooms/275344,2.5 shared baths,Private room,NaN,1.0


In [49]:
import numpy as np

# Define a function to extract the number of baths from the text
def extract_baths(text):
    if pd.isnull(text):
        return np.nan
    if 'half-bath' in text.lower():
        return 0.5
    try:
        # Try to convert the first word to a float (this will work if the text starts with a number)
        return float(text.split()[0])
    except ValueError:
        # If the conversion fails, return NaN
        return np.nan

# Apply the function to the 'bathrooms_text' column and create a new 'baths' column
data['baths'] = data['bathrooms_text'].apply(extract_baths)

# Show the results
data[['bathrooms_text', 'baths']]

,bathrooms_text,baths
0,1 private bath,1.0
1,Shared half-bath,0.5
2,Shared half-bath,0.5
3,2 shared baths,2.0
4,2.5 shared baths,2.5
...,...,...
3436,1 bath,1.0
3437,2 baths,2.0
3438,1 bath,1.0
3439,1.5 baths,1.5


In [50]:
# Check rows where 'baths' is NaN in the data[['bathrooms_text', 'baths']] subset
nan_rows = data[['bathrooms_text', 'baths']][data['baths'].isna()]
nan_rows

,bathrooms_text,baths
26,NaN,NaN
362,NaN,NaN
560,NaN,NaN
695,NaN,NaN
1051,NaN,NaN
1314,NaN,NaN
1862,NaN,NaN
2275,NaN,NaN


In [51]:
# Fill NaN values in 'baths' column with 0
data['baths'].fillna(0, inplace=True)

# Check if the NaN values have been replaced
nan_rows_after_filling = data[['bathrooms_text', 'baths']][data['baths'].isna()]
nan_rows_after_filling

,bathrooms_text,baths


In [52]:
# Define a function to extract bath type from the text
def extract_bath_type(text):
    if pd.isnull(text):
        return np.nan
    text_lower = text.lower()
    if 'private' in text_lower:
        return 'private'
    elif 'shared' in text_lower:
        return 'shared'
    return np.nan

# Apply the function to the 'bathrooms_text' column and create a new 'bath_type' column
data['bath_type'] = data['bathrooms_text'].apply(extract_bath_type)

# Show the results
data[['bathrooms_text', 'bath_type']].head()

,bathrooms_text,bath_type
0,1 private bath,private
1,Shared half-bath,shared
2,Shared half-bath,shared
3,2 shared baths,shared
4,2.5 shared baths,shared


In [53]:
# Check rows where 'bath_type' is NaN in the data[['bathrooms_text', 'bath_type']] subset
nan_rows_bath_type = data[['bathrooms_text', 'bath_type']][data['bath_type'].isna()]
nan_rows_bath_type

,bathrooms_text,bath_type
5,3 baths,NaN
9,1 bath,NaN
11,0 baths,NaN
13,1 bath,NaN
14,2 baths,NaN
...,...,...
3436,1 bath,NaN
3437,2 baths,NaN
3438,1 bath,NaN
3439,1.5 baths,NaN


In [54]:

# Define a function to fill the missing values in 'bath_type' based on the given rules
def fill_missing_bath_type(row):
    if pd.notnull(row['bath_type']):
        return row['bath_type']
    if row['room_type'] == 'Entire home/apt':
        return 'private'
    if pd.isnull(row['bathrooms_text']):
        return 'shared'
    if row['room_type'].lower().startswith('hotel'):
        if row['baths'] == 1:
            return 'private'
        elif row['baths'] > 1:
            return 'shared'
    return np.nan

# Apply the function to fill missing values in 'bath_type'
data['bath_type'] = data.apply(fill_missing_bath_type, axis=1)

# Check if there are still any missing values in 'bath_type'
nan_rows_bath_type_after_filling = data[['room_type', 'bathrooms_text', 'baths', 'bath_type']][data['bath_type'].isna()]
nan_rows_bath_type_after_filling

,room_type,bathrooms_text,baths,bath_type
5,Private room,3 baths,3.0,NaN
9,Private room,1 bath,1.0,NaN
11,Private room,0 baths,0.0,NaN
13,Private room,1 bath,1.0,NaN
14,Private room,2 baths,2.0,NaN
...,...,...,...,...
3429,Private room,1.5 baths,1.5,NaN
3430,Private room,1 bath,1.0,NaN
3436,Private room,1 bath,1.0,NaN
3438,Private room,1 bath,1.0,NaN


In [55]:
# Calculate the percentage of missing values in 'bath_type' for each room type
missing_values_stats = data[data['bath_type'].isna()].groupby('room_type').size() / data.groupby('room_type').size() * 100

# Display the results
missing_values_stats.sort_values(ascending=False)

room_type
Private room       17.768344
Hotel room          1.562500
Entire home/apt          NaN
Shared room              NaN
dtype: float64

In [56]:
# Filter rows where 'room_type' is 'hotel' and 'bath_type' is NaN
hotel_missing_bath_type = data[
    (data['room_type'].str.lower().str.startswith('hotel')) &
    (data['bath_type'].isna())
]
# Display the details of the rows where 'room_type' is 'Hotel room' and 'bath_type' is NaN
hotel_missing_bath_type[['room_type', 'bathrooms_text', 'baths', 'bath_type']]

,room_type,bathrooms_text,baths,bath_type
48,Hotel room,0 baths,0.0,NaN
49,Hotel room,0 baths,0.0,NaN


In [57]:
# Correcting the code to filter 'Hotel room' properly and fill missing 'bath_type' values with 'shared'
data.loc[(data['room_type'].str.lower() == 'hotel room') & (data['bath_type'].isna()), 'bath_type'] = 'shared'

# Check if the values have been updated
hotel_missing_bath_type_updated = data[(data['room_type'].str.lower() == 'hotel room') & (data['bath_type'].isna())]
hotel_missing_bath_type_updated[['room_type', 'bathrooms_text', 'baths', 'bath_type']]

,room_type,bathrooms_text,baths,bath_type


In [58]:
# Fill the 'bath_type' column based on the conditions for 'Private room'
data.loc[(data['room_type'].str.lower() == 'private room') & (data['baths'] <= 1.5), 'bath_type'] = 'private'
data.loc[(data['room_type'].str.lower() == 'private room') & (data['baths'] > 1.5), 'bath_type'] = 'shared'

# Check if the values have been updated
private_room_bath_type_updated = data[data['room_type'].str.lower() == 'private room']
private_room_bath_type_updated[['room_type', 'bathrooms_text', 'baths', 'bath_type']].head()

,room_type,bathrooms_text,baths,bath_type
0,Private room,1 private bath,1.0,private
1,Private room,Shared half-bath,0.5,private
2,Private room,Shared half-bath,0.5,private
3,Private room,2 shared baths,2.0,shared
4,Private room,2.5 shared baths,2.5,shared


In [59]:
# Check if there are any NaN values in the 'bath_type' column for 'Private room'
private_room_nan_bath_type = private_room_bath_type_updated[private_room_bath_type_updated['bath_type'].isna()]
private_room_nan_bath_type[['room_type', 'bathrooms_text', 'baths', 'bath_type']]

,room_type,bathrooms_text,baths,bath_type


In [60]:
import re

# Define a function to extract the number of bedrooms from the 'name' column
def extract_bedrooms(name):
    # Search for a pattern that captures a number followed by 'bedroom' or 'bedrooms'
    match = re.search(r'(\d+(\.\d+)?)\s*bedrooms?', name, flags=re.IGNORECASE)
    if match:
        return float(match.group(1))
    return np.nan

# Apply the function to the 'name' column and create a new 'bedroom' column
data['bedroom'] = data['name'].apply(extract_bedrooms)

# Show the results
data[['name', 'bedroom']].head()


,name,bedroom
0,Villa in Singapore · ★4.44 · 2 bedrooms · 3 be...,2.0
1,Home in Singapore · ★4.16 · 1 bedroom · 1 bed ...,1.0
2,Home in Singapore · ★4.41 · 1 bedroom · 2 beds...,1.0
3,Rental unit in Singapore · ★4.40 · 1 bedroom ·...,1.0
4,Rental unit in Singapore · ★4.54 · 1 bedroom ·...,1.0


In [61]:
# Check rows where 'bedroom' is NaN in the data[['name', 'bedroom']] subset
nan_rows_bedroom = data[['name', 'bedroom']][data['bedroom'].isna()]
nan_rows_bedroom


,name,bedroom
87,Rental unit in Singapore · ★4.22 · Studio · 2 ...,NaN
88,Aparthotel in Singapore · ★4.80 · Studio · 1 b...,NaN
91,Aparthotel in Singapore · ★4.79 · Studio · 1 b...,NaN
104,Aparthotel in Singapore · ★4.71 · Studio · 1 b...,NaN
107,Serviced apartment in Singapore · ★4.69 · Stud...,NaN
...,...,...
3140,Aparthotel in Singapore · Studio · 1 bed · 1 bath,NaN
3277,Boat in Singapore · ★New · 6 beds · 2 baths,NaN
3285,Rental unit in Singapore · ★New · Studio · 1 b...,NaN
3340,Condo in Singapore · ★5.0 · Studio · 1.5 share...,NaN


In [62]:
# Filter rows where 'name' contains 'Studio' and 'bedroom' is not NaN
studio_not_nan = data[(data['name'].str.contains('Studio', case=False, na=False)) & (~data['bedroom'].isna())]

# Display the results
studio_not_nan[['name', 'bedroom']]


,name,bedroom


In [63]:
# Fill 'bedroom' with 1 where 'name' contains 'Studio' and 'bedroom' is NaN
data.loc[(data['name'].str.contains('Studio', case=False, na=False)) & (data['bedroom'].isna()), 'bedroom'] = 1

# Check if the values have been updated
studio_nan_updated = data[(data['name'].str.contains('Studio', case=False, na=False)) & (data['bedroom'].isna())]
studio_nan_updated[['name', 'bedroom']]


,name,bedroom


In [64]:
# Filter rows where 'name' contains 'Studio' and 'bedroom' is not NaN
studio_not_nan = data[(data['name'].str.contains('Studio', case=False, na=False)) & (~data['bedroom'].isna())]

# Display the results
studio_not_nan[['name', 'bedroom']]


,name,bedroom
87,Rental unit in Singapore · ★4.22 · Studio · 2 ...,1.0
88,Aparthotel in Singapore · ★4.80 · Studio · 1 b...,1.0
91,Aparthotel in Singapore · ★4.79 · Studio · 1 b...,1.0
104,Aparthotel in Singapore · ★4.71 · Studio · 1 b...,1.0
107,Serviced apartment in Singapore · ★4.69 · Stud...,1.0
...,...,...
3081,Hotel in Singapore · Studio · 2 beds · 1 priva...,1.0
3140,Aparthotel in Singapore · Studio · 1 bed · 1 bath,1.0
3285,Rental unit in Singapore · ★New · Studio · 1 b...,1.0
3340,Condo in Singapore · ★5.0 · Studio · 1.5 share...,1.0


In [65]:
# Filter rows where 'bedrooms' is not NaN and 'bedrooms' is not equal to 'bedroom'
bedrooms_not_nan_mismatch = data[(~data['bedrooms'].isna()) & (data['bedroom'] != data['bedrooms'])]

# Display the results
bedrooms_not_nan_mismatch[['name', 'bedroom', 'bedrooms']]


,name,bedroom,bedrooms


In [66]:
# Filter rows where 'bedroom' is NaN
bedroom_nan_rows = data[data['bedroom'].isna()]
# Display the results including 'room_type'
bedroom_nan_rows[['name', 'room_type', 'bedroom']]


,name,room_type,bedroom
391,Tiny home in Singapore · ★4.76 · 3 beds · 1 bath,Entire home/apt,NaN
1578,Boat in Singapore · 11 beds · 1 bath,Entire home/apt,NaN
2892,Boat in Singapore · 5 beds · 2 baths,Entire home/apt,NaN
3277,Boat in Singapore · ★New · 6 beds · 2 baths,Entire home/apt,NaN


In [ ]:
# Fill NaN values in 'bedroom' column with 1 for the specified rows
data.loc[data['bedroom'].isna(), 'bedroom'] = 1

# Check if the NaN values have been replaced
bedroom_nan_rows_after_filling = data.loc[bedroom_nan_rows.index, ['name', 'room_type', 'bedroom']]
bedroom_nan_rows_after_filling


In [ ]:
# Drop the 'bedrooms' column
data = data.drop(columns=['bedrooms'])

# Check for rows that contain NaN values
nan_rows_after_dropping = data[data.isna().any(axis=1)]
nan_rows_after_dropping


In [ ]:
# Drop the 'bathrooms_text' column
data = data.drop(columns=['bathrooms_text'])

# Check for rows that contain NaN values
nan_rows_after_dropping_bathrooms_text = data[data.isna().any(axis=1)]
nan_rows_after_dropping_bathrooms_text


In [ ]:
# Define a function to extract the number of beds from the 'name' column, excluding 'bedroom' or 'bedrooms'
def extract_beds_corrected(name):
    # Search for a pattern that captures a number followed by 'bed' or 'beds', but not 'bedroom' or 'bedrooms'
    match = re.search(r'(?<!bed)\b(\d+(\.\d+)?)\s*beds?\b', name, flags=re.IGNORECASE)
    if match:
        return float(match.group(1))
    return np.nan

# Apply the function to the 'name' column and update the 'bed' column
data['bed'] = data['name'].apply(extract_beds_corrected)

# Filter rows where 'beds' is not NaN and 'beds' is not equal to 'bed' after the correction
beds_not_nan_mismatch_corrected = data[(~data['beds'].isna()) & (data['bed'] != data['beds'])]

# Display the results
beds_not_nan_mismatch_corrected[['name', 'bed', 'beds']]

In [ ]:
# Filter rows where 'bed' is NaN
bed_nan_rows = data[data['bed'].isna()]

# Display the results
bed_nan_rows[['name', 'bed']]


In [ ]:
# Fill NaN values in 'bed' column with the corresponding values from 'bedroom' column
data.loc[data['bed'].isna(), 'bed'] = data['bedroom']

# Check if the NaN values in 'bed' column have been replaced
bed_nan_rows_after_filling = data.loc[bed_nan_rows.index, ['name', 'bedroom', 'bed']]
bed_nan_rows_after_filling.head()


In [ ]:
# Fill NaN values in 'bed' column with the corresponding values from 'bedroom' column
data.loc[data['bed'].isna(), 'bed'] = data['bedroom']

# Check if the NaN values in 'bed' column have been replaced
bed_nan_rows_after_filling = data.loc[bed_nan_rows.index, ['name', 'bedroom', 'bed']]
bed_nan_rows_after_filling.head()


In [ ]:
# Drop the 'beds' column
data = data.drop(columns=['beds'])

# Check for rows that contain NaN values in 'bed', 'baths', 'bath_type', and 'bedroom' columns
nan_rows_selected_columns = data[data[['bed', 'baths', 'bath_type', 'bedroom']].isna().any(axis=1)]
nan_rows_selected_columns[['bed', 'baths', 'bath_type', 'bedroom']]


### Data Processing Steps and Logic Summary:
1. Filling the "baths" Column:

- Extract the number from the "bathrooms_text" column as the value for the "baths" column.
- For cases with "half-bath" or "Half-bath", fill in 0.5 in the "baths" column.
- Fill missing values in the "baths" column with 0.
2. Filling the "bath_type" Column:

- Extract "private" or "shared" from the "bathrooms_text" column into the "bath_type" column.
- For rows where "room_type" is "Entire home/apt", fill "private" in the "bath_type" column.
- For rows where "bathrooms_text" is missing, fill "shared" in the "bath_type" column.
- For rows where "room_type" is "Hotel room" and "baths" is 1 or less, fill "private" in the "bath_type" column.
- For rows where "room_type" is "Hotel room" and "baths" is more than 1, fill "shared" in the "bath_type" column.
- For rows where "room_type" is "Private room" and "baths" is 1.5 or less, fill "private" in the "bath_type" column.
- For rows where "room_type" is "Private room" and "baths" is more than 1.5, fill "shared" in the "bath_type" column.
3. Filling the "bedroom" Column:

- Extract the number of bedrooms from the "name" column.
- For rows where "name" contains "Studio", fill 1 in the "bedroom" column.
- For rows where "bedroom" is still missing, fill 1 in the "bedroom" column as these rows do not provide information about bedrooms in the "name" column.
4. Filling the "bed" Column:

- Extract the number of beds from the "name" column, excluding cases of "bedroom" or "bedrooms".
- For rows where "bed" is missing, fill the value from the "bedroom" column.
5. Dropping Unnecessary Columns:

- Drop the "bedrooms", "bathrooms_text", and "beds" columns.
6. Checking for Rows with Missing Values:

- Check for rows that contain missing values in the "bed", "baths", "bath_type", and "bedroom" columns.

In [1]:
import pandas as pd
import re

# 1. Importing the Data
data = pd.read_csv('./csv_output/info_raw.csv')

# 2. Filling the "baths" Column
data['baths'] = data['bathrooms_text'].str.extract(r'(\d*\.?\d+)').astype(float)
data['baths'].fillna(0, inplace=True)
data.loc[data['bathrooms_text'].str.lower().str.contains('half-bath', na=False), 'baths'] = 0.5

# 3. Filling the "bath_type" Column
data['bath_type'] = data['bathrooms_text'].str.extract(r'(private|shared)', flags=re.IGNORECASE)
data.loc[data['room_type'] == 'Entire home/apt', 'bath_type'] = 'private'
data.loc[data['bathrooms_text'].isna(), 'bath_type'] = 'shared'
data.loc[(data['room_type'] == 'Hotel room') & (data['baths'] <= 1), 'bath_type'] = 'private'
data.loc[(data['room_type'] == 'Hotel room') & (data['baths'] > 1), 'bath_type'] = 'shared'
data.loc[(data['room_type'] == 'Private room') & (data['baths'] <= 1.5), 'bath_type'] = 'private'
data.loc[(data['room_type'] == 'Private room') & (data['baths'] > 1.5), 'bath_type'] = 'shared'

# 4. Filling the "bedroom" Column
data['bedroom'] = data['name'].str.extract(r'(\d+(?:\.\d+)?)\s*bedrooms?', flags=re.IGNORECASE).astype(float)
data.loc[data['name'].str.contains('Studio', case=False, na=False), 'bedroom'] = 1
data.loc[data['bedroom'].isna(), 'bedroom'] = 1

# 5. Filling the "bed" Column
data['bed'] = data['name'].str.extract(r'(?<!bed)\b(\d+(?:\.\d+)?)\s*beds?\b', flags=re.IGNORECASE).astype(float)
data['bed'].fillna(data['bedroom'], inplace=True)

# 6. Dropping Unnecessary Columns
data = data.drop(columns=['bedrooms', 'bathrooms_text', 'beds'])

# 7. Checking for Rows with Missing Values
nan_rows_selected_columns = data[data[['bed', 'baths', 'bath_type', 'bedroom']].isna().any(axis=1)]

# 8. Display Results
nan_rows_selected_columns[['bed', 'baths', 'bath_type', 'bedroom']]

,bed,baths,bath_type,bedroom


In [2]:
data.to_csv('./csv_output/info_processed.csv')

In [3]:
# Re-importing pandas package
import pandas as pd

# Loading the datasets again
final_data = pd.read_csv('./dataSource/final_data.csv')
info_processed = pd.read_csv('./csv_output/info_processed.csv')

In [4]:
# Renaming columns in info_processed
info_processed_renamed = info_processed.rename(columns={
    'bedroom': 'total_bedrooms',
    'bed': 'total_beds',
    'baths': 'total_baths',
    'bath_type': 'bath_type'
})

# Merging the datasets on 'id'
merged_data = pd.merge(final_data, info_processed_renamed[['id', 'total_bedrooms', 'total_beds', 'total_baths', 'bath_type']], on='id')
merged_data.head()

,id,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,room_type,price,minimum_months,maximum_months,distance_to_mrt,...,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,listing_url,total_bedrooms,total_beds,total_baths,bath_type
0,71609,Tampines,East Region,1.34537,103.95887,Private room,4170,3,37,0.496792,...,4.00,4.63,4.78,4.26,4.32,https://www.airbnb.com/rooms/71609,2.0,3.0,1.0,private
1,71896,Tampines,East Region,1.34754,103.95958,Private room,2100,3,37,0.678619,...,4.09,4.43,4.43,4.17,4.04,https://www.airbnb.com/rooms/71896,1.0,1.0,0.5,private
2,71903,Tampines,East Region,1.34531,103.96100,Private room,2430,3,37,0.400694,...,4.52,4.63,4.64,4.50,4.36,https://www.airbnb.com/rooms/71903,1.0,2.0,0.5,private
3,275343,Bukit Merah,Central Region,1.29015,103.80814,Private room,1650,3,33,0.580759,...,4.26,4.47,4.42,4.53,4.63,https://www.airbnb.com/rooms/275343,1.0,1.0,2.0,shared
4,275344,Bukit Merah,Central Region,1.28836,103.81144,Private room,2070,2,33,0.612490,...,4.21,4.64,4.57,4.64,4.43,https://www.airbnb.com/rooms/275344,1.0,1.0,2.5,shared


In [5]:
merged_data.to_csv('./dataSource/final_data_5002.csv')